### Carga de datos, etc


In [1]:
import os
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import PIL.Image as Image

from skimage import io, color, transform
from skimage.feature import hog

import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

In [2]:
gray_matrix = np.load('DatosDF\gray_matrix.npy', allow_pickle=True)
hog = np.load('DatosDF\hog.npy', allow_pickle=True)
estados = np.load('DatosDF\estados.npy', allow_pickle=True)
label = np.load('DatosDF\labels.npy', allow_pickle=True)

<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\e'
<>:4: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\e'
<>:4: SyntaxWarning: invalid escape sequence '\l'
C:\Users\gonza\AppData\Local\Temp\ipykernel_14624\488994593.py:1: SyntaxWarning: invalid escape sequence '\g'
  gray_matrix = np.load('DatosDF\gray_matrix.npy', allow_pickle=True)
C:\Users\gonza\AppData\Local\Temp\ipykernel_14624\488994593.py:2: SyntaxWarning: invalid escape sequence '\h'
  hog = np.load('DatosDF\hog.npy', allow_pickle=True)
C:\Users\gonza\AppData\Local\Temp\ipykernel_14624\488994593.py:3: SyntaxWarning: invalid escape sequence '\e'
  estados = np.load('DatosDF\estados.npy', allow_pickle=True)
C:\Users\gonza\AppData\Local\Temp\ipykernel_14624\488994593.py:4: SyntaxWarning: inv

In [3]:
df = pd.DataFrame({'gray_matrix': gray_matrix,'hog': hog,'estados': estados,'label': label})

In [4]:
dfxusar = df[df['estados'] == 1]
elresto = df[df['estados'] == 0]
dfxusar

,gray_matrix,hog,estados,label
16,"[[175, 166, 185, 198, 202, 213, 212, 209, 210,...","[0.20444939, 0.034390625, 0.026585897, 0.01487...",1,angry
23,"[[26, 29, 30, 35, 45, 60, 108, 145, 161, 161, ...","[0.46974203, 0.15378156, 0.0056655, 0.0, 0.002...",1,angry
30,"[[17, 18, 19, 19, 17, 15, 16, 17, 18, 20, 18, ...","[0.16117984, 0.10060929, 0.12280247, 0.0026138...",1,angry
31,"[[0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 13, 37, ...","[0.17848918, 0.014774916, 0.0, 0.0, 0.0, 0.0, ...",1,angry
45,"[[254, 254, 254, 254, 254, 254, 254, 254, 254,...","[0.025827257, 0.01296422, 0.020755643, 0.0, 0....",1,angry
...,...,...,...,...
35863,"[[9, 38, 69, 99, 86, 102, 111, 105, 65, 58, 64...","[0.23134665, 0.3235038, 0.13396424, 0.05265233...",1,surprise
35869,"[[253, 253, 254, 253, 170, 101, 105, 102, 85, ...","[0.4190711, 0.32569548, 0.0045682807, 0.002312...",1,surprise
35875,"[[252, 247, 146, 93, 94, 89, 99, 94, 93, 83, 9...","[0.32172725, 0.29283354, 0.09000543, 0.0720573...",1,surprise
35878,"[[33, 44, 49, 40, 35, 28, 23, 14, 9, 16, 17, 5...","[0.20763314, 0.31089163, 0.1581842, 0.02511550...",1,surprise


In [5]:
X = np.array(dfxusar['hog'].values.tolist())
y = dfxusar['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19011)

# Redes Neuronales

In [9]:
mlp = MLPClassifier(hidden_layer_sizes=(100,),
                    activation='logistic',
                    solver='sgd',
                    alpha=0,
                    learning_rate='constant',
                    max_iter=10000,
                    tol=1e-6,
                    verbose=True,
                    momentum=0,
                    n_iter_no_change=25,
                    random_state=42)
mlp.fit(X_train, y_train)

InvalidParameterError: The 'activation' parameter of MLPClassifier must be a str among {'logistic', 'identity', 'relu', 'tanh'}. Got 'softmax' instead.

In [ ]:
&&time
y_pred_e= mlp.predict(X_train)
accuracy = accuracy_score(y_train, y_pred_e)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_train, y_pred_e))

Precisión del modelo: 0.8695770065075922
              precision    recall  f1-score   support

       angry       0.84      0.84      0.84       462
     disgust       0.76      0.67      0.71       435
        fear       0.82      0.78      0.80       579
       happy       0.95      0.99      0.97       650
     neutral       0.93      0.94      0.94       622
         sad       0.80      0.85      0.82       467
    surprise       0.92      0.95      0.94       473

    accuracy                           0.87      3688
   macro avg       0.86      0.86      0.86      3688
weighted avg       0.87      0.87      0.87      3688

In [ ]:
# Realizar predicciones en el conjunto de prueba
y_pred = mlp.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_test, y_pred))

Precisión del modelo: 0.7908992416034669
              precision    recall  f1-score   support

       angry       0.67      0.69      0.68       101
     disgust       0.56      0.48      0.52       112
        fear       0.73      0.77      0.75       156
       happy       0.92      0.97      0.94       149
     neutral       0.89      0.88      0.88       153
         sad       0.73      0.74      0.74       119
    surprise       0.92      0.89      0.90       133

    accuracy                           0.79       923
   macro avg       0.77      0.78      0.77       923
weighted avg       0.79      0.79      0.79       923



## [Conjunto Pequeño] GridSearch - Optimizacion de parametros

### Primero

In [ ]:
%%time

# Define el modelo de Red Neuronal
mlp = MLPClassifier(hidden_layer_sizes=(100,),
                    activation='logistic',
                    solver='sgd',
                    alpha=0,
                    learning_rate='constant',
                    max_iter=10000,
                    tol=1e-4,
                    verbose=True,
                    momentum=0,
                    n_iter_no_change=25,
                    random_state=42)

# Define una cuadrícula de hiperparámetros que deseas explorar
param_grid = {
    'max_iter': [500,1000,2000],
    'learning_rate_init': [0.1,0.001,0.0001],
    'batch_size': [32,128,512]
}

# Realiza la búsqueda en cuadrícula con validación cruzada
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=5, scoring='accuracy', verbose=10,n_jobs=8)
grid_search.fit(X_train, y_train)

# Muestra los mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

# Entrena el modelo con los mejores hiperparámetros en todo el conjunto de entrenamiento
best_rf_classifier = grid_search.best_estimator_
best_rf_classifier.fit(X_train, y_train)

# Evalúa el modelo en el conjunto de prueba
y_pred = best_rf_classifier.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_test, y_pred))

Mejores hiperparámetros: 
- 'batch_size': 512
- 'learning_rate_init': 0.1
- 'max_iter': 1000

Precisión del modelo: 0.8266522210184182
              precision    recall  f1-score   support

       angry       0.72      0.78      0.75       101
     disgust       0.70      0.61      0.65       112
        fear       0.80      0.83      0.81       156
       happy       0.97      0.98      0.97       149
     neutral       0.86      0.88      0.87       153
         sad       0.72      0.72      0.72       119
    surprise       0.93      0.91      0.92       133

    accuracy                           0.83       923
   macro avg       0.82      0.81      0.81       923\
weighted avg       0.83      0.83      0.83       923\

CPU times: total: 13min 54s\
Wall time: 3h 22min 48s

### Segundo - con conocimiento de causa

In [10]:
%%time

# Define el modelo de Red Neuronal
mlp = MLPClassifier(hidden_layer_sizes=(100,),
                    activation='logistic',
                    solver='sgd',
                    alpha=0,
                    learning_rate='constant',
                    max_iter=10000,
                    tol=1e-4,
                    verbose=True,
                    momentum=0,
                    early_stopping=True,
                    validation_fraction = 0.2,
                    n_iter_no_change=25,
                    random_state=42)

# Define una cuadrícula de hiperparámetros que deseas explorar
param_grid = {
    'learning_rate_init': [0.2,0.1,0.01],
    'batch_size': [32,128,512],
    'activation' : ['identity', 'logistic', 'tanh', 'relu'],
    'hidden_layer_sizes' : [(50,), (100,), (150,), (50, 50), (100, 100), (150, 150)]
}

# Realiza la búsqueda en cuadrícula con validación cruzada
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=5, scoring='accuracy', verbose=10,n_jobs=12)
grid_search.fit(X_train, y_train)

# Muestra los mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

# Entrena el modelo con los mejores hiperparámetros en todo el conjunto de entrenamiento
best_rf_classifier = grid_search.best_estimator_
best_rf_classifier.fit(X_train, y_train)

# Evalúa el modelo en el conjunto de prueba
y_pred = best_rf_classifier.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


C:\Users\gonza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
30 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\gonza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\gonza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-pa

Iteration 1, loss = 1.63129127
Validation score: 0.326558
Iteration 2, loss = 1.12056612
Validation score: 0.288618
Iteration 3, loss = 0.94868607
Validation score: 0.472900
Iteration 4, loss = 0.76849252
Validation score: 0.712737
Iteration 5, loss = 0.64239932
Validation score: 0.739837
Iteration 6, loss = 0.58789145
Validation score: 0.745257
Iteration 7, loss = 0.50836185
Validation score: 0.769648
Iteration 8, loss = 0.44111627
Validation score: 0.760163
Iteration 9, loss = 0.36683207
Validation score: 0.710027
Iteration 10, loss = 0.37076293
Validation score: 0.827913
Iteration 11, loss = 0.32760500
Validation score: 0.825203
Iteration 12, loss = 0.35472833
Validation score: 0.411924
Iteration 13, loss = 0.58741363
Validation score: 0.643631
Iteration 14, loss = 0.40432416
Validation score: 0.520325
Iteration 15, loss = 0.28786891
Validation score: 0.509485
Iteration 16, loss = 0.26738734
Validation score: 0.826558
Iteration 17, loss = 0.18056639
Validation score: 0.776423
Iterat

**Resultados** 

Precisión del modelo: 0.8266522210184182
- 'activation': 'relu', 
- 'batch_size': 32, 
- 'hidden_layer_sizes': (150, 150), 
- 'learning_rate_init': 0.1

### Tercero 

In [11]:
%%time

# Define el modelo de Red Neuronal
mlp = MLPClassifier(hidden_layer_sizes=(100,),
                    activation='relu',
                    solver='sgd',
                    alpha=0,
                    learning_rate='constant',
                    max_iter=10000,
                    tol=1e-4,
                    verbose=True,
                    momentum=0,
                    early_stopping=True,
                    validation_fraction = 0.2,
                    n_iter_no_change=25,
                    random_state=42)

# Define una cuadrícula de hiperparámetros que deseas explorar
param_grid = {
    'learning_rate_init': [0.15,0.1,0.05],
    'batch_size': [16,32,64],
    'hidden_layer_sizes' : [(200,), (150, 150), (200, 200), (50, 50, 50), (100, 100, 100), (150, 150,150)]
}

# Realiza la búsqueda en cuadrícula con validación cruzada
grid_search_thethird = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=5, scoring='accuracy', verbose=10,n_jobs=12)
grid_search_thethird.fit(X_train, y_train)

# Muestra los mejores hiperparámetros encontrados
best_params = grid_search_thethird.best_params_
print(f"Mejores hiperparámetros: {best_params}")

# Entrena el modelo con los mejores hiperparámetros en todo el conjunto de entrenamiento
best_rf_classifier = grid_search_thethird.best_estimator_
best_rf_classifier.fit(X_train, y_train)

# Evalúa el modelo en el conjunto de prueba
y_pred = best_rf_classifier.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Iteration 1, loss = 1.66731520
Validation score: 0.517615
Iteration 2, loss = 1.17159039
Validation score: 0.677507
Iteration 3, loss = 0.90054906
Validation score: 0.677507
Iteration 4, loss = 0.79441614
Validation score: 0.528455
Iteration 5, loss = 0.66462087
Validation score: 0.691057
Iteration 6, loss = 0.55706461
Validation score: 0.609756
Iteration 7, loss = 0.48991086
Validation score: 0.764228
Iteration 8, loss = 0.46645658
Validation score: 0.661247
Iteration 9, loss = 1.01795763
Validation score: 0.371274
Iteration 10, loss = 0.56822382
Validation score: 0.707317
Iteration 11, loss = 0.47515479
Validation score: 0.650407
Iteration 12, loss = 0.36321238
Validation score: 0.566396
Iteration 13, loss = 0.34837084
Validation score: 0.779133
Iteration 14, loss = 0.36646717
Validation score: 0.757453
Iteration 15, loss = 0.22720211
Validation score: 0.823848
Iteration 16, loss = 0.19869274
Validation score: 0.817073
Ite

**Resultados** 

Precisión del modelo: 0.8331527627302275
- 'activation': 'relu', 
- 'batch_size': 16, 
- 'hidden_layer_sizes': (150, 150,150), 
- 'learning_rate_init': 0.1

## [Conjunto Grande] GridSearch - Optimizacion de parametros

Modelo aplicando los parametros optimizados

In [6]:
X_g = np.array(df['hog'].values.tolist())
y_g = df['label']
X_traing, X_testg, y_traing, y_testg = train_test_split(X_g, y_g, test_size=0.2, random_state=19011)

In [7]:
mlp = MLPClassifier(hidden_layer_sizes=(100,),
                    activation='logistic',
                    solver='sgd',
                    alpha=0,
                    learning_rate='constant',
                    learning_rate_init= 0.1,
                    batch_size=512,
                    max_iter=1000,
                    tol=1e-6,
                    verbose=True,
                    momentum=0,
                    n_iter_no_change=25,
                    random_state=42)
mlp.fit(X_traing, y_traing)

Iteration 1, loss = 1.81516160
Iteration 2, loss = 1.79452661
Iteration 3, loss = 1.78041330
Iteration 4, loss = 1.76372188
Iteration 5, loss = 1.74462249
Iteration 6, loss = 1.72290255


C:\Users\gonza\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(activation='logistic', alpha=0, batch_size=512,
              learning_rate_init=0.1, max_iter=1000, momentum=0,
              n_iter_no_change=25, random_state=42, solver='sgd', tol=1e-06,
              verbose=True)

In [10]:

y_pred_eg= mlp.predict(X_traing)
accuracy = accuracy_score(y_traing, y_pred_eg)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_traing, y_pred_eg))

Precisión del modelo: 0.7871050889964819
              precision    recall  f1-score   support

       angry       0.98      0.47      0.63      3905
     disgust       0.93      0.74      0.83       438
        fear       0.72      0.81      0.76      4106
       happy       0.79      0.98      0.87      7238
     neutral       0.88      0.71      0.79      4928
         sad       0.89      0.69      0.78      4864
    surprise       0.63      0.99      0.77      3230

    accuracy                           0.79     28709
   macro avg       0.83      0.77      0.77     28709
weighted avg       0.82      0.79      0.78     28709



In [11]:
# Realizar predicciones en el conjunto de prueba
y_predg = mlp.predict(X_testg)

# Evaluar el modelo
accuracy = accuracy_score(y_testg, y_predg)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_testg, y_predg))

Precisión del modelo: 0.48091390359431596
              precision    recall  f1-score   support

       angry       0.55      0.20      0.29      1048
     disgust       0.55      0.40      0.47       109
        fear       0.32      0.37      0.34      1015
       happy       0.60      0.80      0.68      1751
     neutral       0.50      0.37      0.42      1270
         sad       0.41      0.30      0.35      1213
    surprise       0.43      0.76      0.55       772

    accuracy                           0.48      7178
   macro avg       0.48      0.46      0.44      7178
weighted avg       0.48      0.48      0.46      7178



### Primero

In [7]:
%%time

# Define el modelo de Red Neuronal
mlp = MLPClassifier(hidden_layer_sizes=(100,),
                    activation='logistic',
                    solver='sgd',
                    alpha=0,
                    learning_rate='constant',
                    max_iter=10000,
                    tol=1e-4,
                    verbose=True,
                    momentum=0,
                    n_iter_no_change=25,
                    random_state=42)

# Define una cuadrícula de hiperparámetros que deseas explorar
param_grid = {
    'max_iter': [500,1000,1500],
    'learning_rate_init': [0.1,0.2],
    'batch_size': [32,128,512,1024]
}

# Realiza la búsqueda en cuadrícula con validación cruzada
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=5, scoring='accuracy', verbose=10,n_jobs=12)
grid_search.fit(X_traing, y_traing)

# Muestra los mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Iteration 1, loss = 1.65838090
Iteration 2, loss = 1.49363521
Iteration 3, loss = 1.43291446
Iteration 4, loss = 1.39118319
Iteration 5, loss = 1.36168545
Iteration 6, loss = 1.33866089
Iteration 7, loss = 1.31643181
Iteration 8, loss = 1.29561440
Iteration 9, loss = 1.27507657
Iteration 10, loss = 1.25390170
Iteration 11, loss = 1.23407896
Iteration 12, loss = 1.21525411
Iteration 13, loss = 1.19428303
Iteration 14, loss = 1.17623807
Iteration 15, loss = 1.15047028
Iteration 16, loss = 1.12946292
Iteration 17, loss = 1.10979209
Iteration 18, loss = 1.08888371
Iteration 19, loss = 1.06735436
Iteration 20, loss = 1.04781276
Iteration 21, loss = 1.02489057
Iteration 22, loss = 1.00832941
Iteration 23, loss = 0.98119081
Iteration 24, loss = 0.95867948
Iteration 25, loss = 0.93541617
Iteration 26, loss = 0.91681697
Iteration 27, loss = 0.89528415
Iteration 28, loss = 0.87432825
Iteration 29, loss = 0.85460498
Iteration 30, loss 

Iteration 251, loss = 0.00873281
Iteration 252, loss = 0.03442232
Iteration 253, loss = 0.00879484
Iteration 254, loss = 0.00868669
Iteration 255, loss = 0.00858415
Iteration 256, loss = 0.00830492
Iteration 257, loss = 0.00818163
Iteration 258, loss = 0.00849371
Iteration 259, loss = 0.00823352
Iteration 260, loss = 0.00838489
Training loss did not improve more than tol=0.000100 for 25 consecutive epochs. Stopping.
Mejores hiperparámetros: {'batch_size': 32, 'learning_rate_init': 0.2, 'max_iter': 500}
CPU times: total: 1h 23min 24s
Wall time: 10h 26min 50s


In [8]:
# Entrena el modelo con los mejores hiperparámetros en todo el conjunto de entrenamiento
best_rf_classifier = grid_search.best_estimator_
best_rf_classifier.fit(X_traing, y_traing)

# Evalúa el modelo en el conjunto de prueba
y_predg = best_rf_classifier.predict(X_testg)

# Evaluar el modelo
accuracy = accuracy_score(y_testg, y_predg)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_testg, y_predg))

Iteration 1, loss = 1.65838090
Iteration 2, loss = 1.49363521
Iteration 3, loss = 1.43291446
Iteration 4, loss = 1.39118319
Iteration 5, loss = 1.36168545
Iteration 6, loss = 1.33866089
Iteration 7, loss = 1.31643181
Iteration 8, loss = 1.29561440
Iteration 9, loss = 1.27507657
Iteration 10, loss = 1.25390170
Iteration 11, loss = 1.23407896
Iteration 12, loss = 1.21525411
Iteration 13, loss = 1.19428303
Iteration 14, loss = 1.17623807
Iteration 15, loss = 1.15047028
Iteration 16, loss = 1.12946292
Iteration 17, loss = 1.10979209
Iteration 18, loss = 1.08888371
Iteration 19, loss = 1.06735436
Iteration 20, loss = 1.04781276
Iteration 21, loss = 1.02489057
Iteration 22, loss = 1.00832941
Iteration 23, loss = 0.98119081
Iteration 24, loss = 0.95867948
Iteration 25, loss = 0.93541617
Iteration 26, loss = 0.91681697
Iteration 27, loss = 0.89528415
Iteration 28, loss = 0.87432825
Iteration 29, loss = 0.85460498
Iteration 30, loss = 0.82399499
Iteration 31, loss = 0.80848128
Iteration 32, los

Iteration 253, loss = 0.00879484
Iteration 254, loss = 0.00868669
Iteration 255, loss = 0.00858415
Iteration 256, loss = 0.00830492
Iteration 257, loss = 0.00818163
Iteration 258, loss = 0.00849371
Iteration 259, loss = 0.00823352
Iteration 260, loss = 0.00838489
Training loss did not improve more than tol=0.000100 for 25 consecutive epochs. Stopping.
Precisión del modelo: 0.5227082752855948
              precision    recall  f1-score   support

       angry       0.46      0.38      0.41      1048
     disgust       0.54      0.48      0.50       109
        fear       0.38      0.36      0.37      1015
       happy       0.69      0.73      0.71      1751
     neutral       0.47      0.47      0.47      1270
         sad       0.41      0.42      0.41      1213
    surprise       0.63      0.72      0.67       772

    accuracy                           0.52      7178
   macro avg       0.51      0.51      0.51      7178
weighted avg       0.52      0.52      0.52      7178

